# RMA

In [1]:
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
def read_task_set_from_txt(file):
    task_set = []
    with open(file, 'r') as txtfile:
        for line in txtfile:
            task = tuple(map(int, line.strip()[1:-1].split(', ')))
            task_set.append(task)
    return task_set

In [3]:
data=read_task_set_from_txt('task_set.txt')
df = pd.DataFrame(data, columns=['id', 'execution', 'arrival', 'period', 'dead'])
df['rem']=df['execution']
df['nextdead']=df['period']
df=df.drop(columns=['dead'])
# Print the DataFrame
print(df)

    id  execution  arrival  period  rem  nextdead
0    1         10        0     255   10       255
1    2         25        0     364   25       364
2    3         24        0     482   24       482
3    4         29        0     324   29       324
4    5         18        0     352   18       352
5    6         15        0     400   15       400
6    7         15        0     237   15       237
7    8         24        0     206   24       206
8    9         13        0     355   13       355
9   10         12        0     299   12       299
10  11         26        0     369   26       369
11  12         10        0     401   10       401
12  13         14        0     399   14       399
13  14         21        0     367   21       367
14  15         20        0     151   20       151


In [4]:
import math
def lcm(a, b):
    return abs(a*b) // math.gcd(a, b)

def hyperperiod(periods):
    h = periods[0]
    for p in periods[1:]:
        h = lcm(h, p)
    return h


In [5]:
import pandas as pd

def check_rma_schedulability(df):
    n = len(df)
    print(n)
    total_utilization = 0

    for index, row in df.iterrows():
        utilization = row['execution'] / row['period']
        total_utilization += utilization

    max_utilization = n * (2 ** (1/n) - 1)

    if total_utilization <= max_utilization:
        print("Schedulable by RMA")
        print(max_utilization)
        print(total_utilization)
        
        return True
    else:
        print("Not schedulable by RMA")
        print(total_utilization)
        print(max_utilization)
        return False

In [6]:
sc=[]
if check_rma_schedulability(df):
    
    for i in range(500):
            task=df[df["arrival"] <= i][df["rem"] > 0]
            if len(task)>0:
                nexttask=task.sort_values('period').index[0]
                df.loc[nexttask, "rem"] -= 1.0
                if i > 0 and sc[-1]['id'] == df.loc[nexttask, "id"] and sc[-1]['end time'] == i:
                    sc[-1]['end time'] += 1
                    sc[-1]['total time'] += 1
                else:
                    dict={'id': df.loc[nexttask, "id"], 'start time': i, 'end time': i + 1, 'total time': 1}
                    sc.append(dict)
            arrived=df[(i+1)%df['period']==0].index
            df.loc[arrived, 'rem'] = df.loc[arrived, 'execution']
            df.loc[arrived, 'nextdead'] = df.loc[arrived, 'nextdead']+df.loc[arrived,'period']
        #print("i=",i)
        #print(df)

15
Not schedulable by RMA
0.9125386830738769
0.7094118423094009


In [7]:
sc1=pd.DataFrame(sc)
sc1

""


In [8]:
if check_rma_schedulability(df):
    import matplotlib.pyplot as plt
    fig, gnt = plt.subplots()
    gnt.set_ylim(0, 50)
    gnt.set_xlim(0, 500)
    gnt.set_xlabel('time')
    gnt.set_ylabel('Tasks')

    tick=10
    tc=[]
    lab=[]
    for i in df['id']:
        tc.append(tick)
        lab.append(i)
        tick+=10

    gnt.set_yticks(tc)
    gnt.set_yticklabels(lab)
    gnt.grid(True)

    for item in sc:
        gnt.broken_barh([(item['start time'], item['end time']-item['start time'])], (10*item['id'], 5), facecolors =('tab:green'))




15
Not schedulable by RMA
0.9125386830738769
0.7094118423094009
